# PyZX interoperability: Pauli webs

*(If you "run all" the notebook and do not see visualisations after the block that runs the main algorithm, run it again. The algorithm is designed to run in series or parallel but, for clarity, this notebook runs it only once. The notebook can fail without this meaning there is something wrong with the algorithm.)*

This notebook offers an example of algorithmic lattice surgery with an emphasis on transfering Pauli webs available in the original PyZX graph to the final result.

This notebook goes over early parts of the process relatively quickly. Check other notebooks for insight into early steps of the process. 

In [ ]:
import os
import sys
import pyzx as zx

from pyzx.pauliweb import compute_pauli_webs
zx.settings.colors = zx.rgb_colors
%matplotlib widget

repository_root = os.path.abspath(os.path.join(os.getcwd(), "..", ".."))
output_folder_path = f"{repository_root}/outputs/txt"
if repository_root not in sys.path:
    sys.path.insert(0, repository_root)
from scripts.runner import runner
from utils.interop_pyzx import get_simple_graph_from_pyzx
from utils.grapher import make_graph_from_final_lattice, visualise_3d_graph



## Input: PyZX graph

Let's start by preparing a PyZX graph. We'll use a graph similar to those in the [PyZX notebook that introduces Pauli Webs](https://nbviewer.org/github/zxcalc/pyzx/blob/master/demos/PauliWebsRGB.ipynb) but a little smaller to increase the odds of a quick 3D placements.

In [ ]:
circuit_name = "pyzx_example_pauli_webs"

c = zx.qasm("""
qreg q[3];
cx q[0], q[1];
cx q[2], q[1];
cx q[0], q[2];
cx q[2], q[1];
""")
pyzx_graph = c.to_graph()

zx.draw(pyzx_graph, labels=True)

Let's now compute and examine all Pauli Webs.

For context, in PyZX, a Pauli web is generated by reference to the boundary in each qubit line, and there are two webs per boundary, X and Z.

In [ ]:
order, zwebs, xwebs = compute_pauli_webs(pyzx_graph)
zx.draw(pyzx_graph, labels=True, pauli_web=xwebs[11])
zx.draw(pyzx_graph, labels=True, pauli_web=zwebs[11])
zx.draw(pyzx_graph, labels=True, pauli_web=xwebs[12])
zx.draw(pyzx_graph, labels=True, pauli_web=zwebs[12])
zx.draw(pyzx_graph, labels=True, pauli_web=xwebs[13])
zx.draw(pyzx_graph, labels=True, pauli_web=zwebs[13])

## Process: Running the algorithm

To pass the graph to the algorithm, we need to distill the PyZX graph into a `simple_graph`, i.e., a simple dictionary of nodes and edges. The algorithm does not need information about Pauli webs to perform its foundational task of placing spiders in the 3D space – we will deal with this later. 

In [ ]:
simple_graph = get_simple_graph_from_pyzx(pyzx_graph)
print(simple_graph)

The next step is to feed the `simple_graph` to the algorithm and let it work its magic (it's not magic, but let's pretend it is).

If/when the algorithm is successful, it will produce:
- an interactive visualisation displayed on screen,
- an animation of the process saved to `./outputs/media/`,
- node-by-node/edge-by-edge text results saved to `./outputs/txt/`,
- and related objects for programmatic use.

It is good practice to examine the results printed to screen, too. In short, the algorithm visits all spiders in a ZX-graph and exchanges them to a 3D version of themselves, and processes edges by also encoding them as a special kind of 3D edge. The printouts on screen detail the order of operations.

**Note!** This notebook limits the number of attempts by the algorithm to a single attempt using the optional `max_attempts=1` parameter. You may need to run the following block a few times to get a successful result. The standard approach is to give the algorithm 10 attempts. The number of attempts is limited here for readability. 

In [ ]:
# Define weights for the value function to to choose best of several valid paths per each edge based on: (length of path, number of beams broken by path)
# A negative value for length of path favours short paths.
# A negative value for number of beams broken by path favours placements that do not block potential open faces requiring connections.
VALUE_FUNCTION_HYPERPARAMS = (-1, -0.5)

# Define a desired length of beams
# The longer the beams, the more space between faces. This feature needs improvement but, in theory, increases the odds of success.
# Needs to be combined with an equal or larger `MAX_PATHFINDER_SEARCH_SPACE`
LENGTH_OF_BEAMS = 3

# Define the maximum size of the search space (distance between source and target node for each pathfinding iteration)
# Larger values will produce a larger number of paths, and longer paths. This increases the odds of a successful result, with a significant impact on runtime.
MAX_PATHFINDER_SEARCH_SPACE = 3

kwargs = {
    "weights": VALUE_FUNCTION_HYPERPARAMS,
    "length_of_beams": LENGTH_OF_BEAMS,
    "max_search_space": MAX_PATHFINDER_SEARCH_SPACE,
}

simple_graph_after_use, edge_paths, lattice_nodes, lattice_edges = runner(
    simple_graph, circuit_name, strip_boundaries=False, hide_boundaries=False, max_attempts=1, visualise="final", **kwargs
)

## Output: Rebuilding Pauli webs.

It is possible to rebuild Pauli webs because there is a relation between the spiders and edges in the PyZX graph and the corresponding 3D blocks in the objects created by the algorithm.
- The algorithm maintains the IDs of any spider in the original graph, using additional/different IDs for intermediate blocks needed to clear any paths.
- The algorithm also notes the original edge for all 3D edges and edge subsegments.

Conceptually, thus, the challenge of rebuilding Pauli Webs comes down to applying each specific Pauli web to the corresponding 3D blocks.
- Easy for short edges: source and target IDs for the edge remain the same as in the original PyZX graph. 
- Hard for long edges: in the final result, some original edges get broken into several segments.

In [ ]:
if lattice_nodes:
    for key, node in lattice_nodes.items():
        print(f"Node ID {key}: {node}.")

if lattice_edges:
    for key, edge in lattice_edges.items():
        print(f"Edge ID {key}: {edge[0]} (corresponding edge in ZX-graph: {edge[1]}).")

So, the first step is to get an edge-by-edge summary of each Pauli web in the original PyZX graph.

In [ ]:
boundaries = [key for key, value in pyzx_graph.types().items() if value == 0]
boundaries = boundaries[pyzx_graph.qubit_count() :]

original_pyzx_z_webs_half_edges = []
original_pyzx_x_webs_half_edges = []

for id in boundaries:
    z_edges_temp = {}
    for key, val in zwebs[id].half_edges().items():
        sorted_key = tuple(sorted(key))
        if sorted_key not in list(z_edges_temp.keys()):
            z_edges_temp[sorted_key] = val
    original_pyzx_z_webs_half_edges.append(z_edges_temp)

    x_edges_temp = {}
    for key, val in xwebs[id].half_edges().items():
        sorted_key = tuple(sorted(key))
        if sorted_key not in list(x_edges_temp.keys()):
            x_edges_temp[sorted_key] = val
    original_pyzx_x_webs_half_edges.append(x_edges_temp)


for edge in original_pyzx_z_webs_half_edges:
    print(edge)
print()
for edge in original_pyzx_x_webs_half_edges:
    print(edge)

Next, we go over the extracted webs and determine if the original edge exists in the 3D version of the ZX graph or if the edge has been broken into segments.

In [ ]:
zwebs_3d = []
xwebs_3d = []

if lattice_edges:

    for original_pyzx_z_webs_edge in original_pyzx_z_webs_half_edges:

        original_pyzx_z_webs_edge_keys_sorted = [
            tuple(sorted(key)) for key in original_pyzx_z_webs_edge.keys()
        ]

        current_web = {}
        for key, lattice_edge in lattice_edges.items():
            original_key_for_lattice_edge = tuple(sorted(lattice_edge[1]))
            if original_key_for_lattice_edge in original_pyzx_z_webs_edge_keys_sorted:
                current_web[key] = original_pyzx_z_webs_edge[
                    original_key_for_lattice_edge
                ]
        zwebs_3d.append(current_web)

    for original_pyzx_x_webs_edge in original_pyzx_x_webs_half_edges:

        original_pyzx_x_webs_edge_keys_sorted = [
            tuple(sorted(key)) for key in original_pyzx_x_webs_edge.keys()
        ]

        current_web = {}
        for key, lattice_edge in lattice_edges.items():
            original_key_for_lattice_edge = tuple(sorted(lattice_edge[1]))
            if original_key_for_lattice_edge in original_pyzx_x_webs_edge_keys_sorted:
                current_web[key] = original_pyzx_x_webs_edge[
                    original_key_for_lattice_edge
                ]
        xwebs_3d.append(current_web)

for i in zwebs_3d:
    print(i)
print()
for i in xwebs_3d:
    print(i)

Finally, we can visualise the webs applied to the 3D space-time diagram produced by the algorithm, and compare them with the webs in the original PyZX graph.

In [ ]:
# X webs
for i, id in enumerate(boundaries):

    print(f"\nVisualising X web {i+1} of {len(boundaries)} (original v. 3D)\n")
    zx.draw(pyzx_graph, labels=True, pauli_web=xwebs[id])
    
    if lattice_nodes and lattice_edges:
        print(xwebs_3d[i])
        final_nx_graph, pauli_webs_graph = make_graph_from_final_lattice(
            lattice_nodes, lattice_edges, xwebs_3d[i]
        )
        visualise_3d_graph(final_nx_graph, pauli_webs_graph=pauli_webs_graph)

# Z webs
for i, id in enumerate(boundaries):

    print(f"\nVisualising Z web {i+1} of {len(boundaries)} (original v. 3D)\n")
    zx.draw(pyzx_graph, labels=True, pauli_web=zwebs[id])
    
    if lattice_nodes and lattice_edges:
        final_nx_graph, pauli_webs_graph = make_graph_from_final_lattice(
            lattice_nodes, lattice_edges, zwebs_3d[i]
        )
        visualise_3d_graph(final_nx_graph, pauli_webs_graph=pauli_webs_graph)

We generated many visualisations. If visualisations seem sluggy or plainly do not show, run the code below to force close.

In [ ]:
#from matplotlib.pyplot import close
#close()